# Notes
* Use 12 months leading for dependent variable
* Financial distress window should be equivalent to dependent variable window
* All models should use only historical data to make prediction, retraining at year end
* Each month, the top 1000 companies should be excluded
* Price calculation should become absolute value
* Forward fill missing values
* Try tests without redundant columns
* Add additional variables
* Scaling independent variables helps with coefficient interpretability
* Dates are end of month (i.e. 3/31/2004 -> 3/2004)

### Initialize

In [1]:
# import libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from statsmodels.api import OLS, add_constant
import matplotlib.pyplot as plt
from statsmodels.tools.sm_exceptions import PerfectSeparationError
import pandas as pd
import numpy as np
import shap

In [3]:
# define constants & assumptions
DEPENDENT_VARIABLE_WINDOW = 12
FINANCIAL_DISTRESS_THRESHOLD = -0.5
MARKET_CAP_PERCENTILE = 0.9
PORTFOLIO_QUANTITY = 10
DEPENDENT_VARIABLE = [f'financialDistress_{DEPENDENT_VARIABLE_WINDOW}_periods']
INDEPENDENT_VARIABLES = ['NITA', 'NIMTA', 'TLTA', 'TLMTA', 'EXRET', 'RSIZE', 'CASHMTA', 'SIGMA']

In [7]:
# load data
df = pd.read_csv('DJJX-data.csv', index_col=0)
df

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


,PERMNO,date_month,GVKEY,cik,conm,atq,ceqq,cheq,ltq,niq,...,NITA,NIMTA,TLTA,TLMTA,RSIZE,CASHMTA,financialDistress,everFinanciallyDistressed,combined,financialDistress_12_periods
0,10000,1986-02,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,...,-0.000110,-0.000011,0.000481,0.000048,-18.705195,0.000062,0,1,0,0
1,10000,1986-03,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,...,-0.000081,-0.000008,0.000352,0.000035,-18.444028,0.000045,0,1,0,0
2,10000,1986-04,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.265,0.689,0.743,0.576,-0.132,...,-0.000087,-0.000009,0.000379,0.000038,-18.500193,0.000049,0,1,0,0
3,10000,1986-05,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,...,-0.000410,-0.000041,0.000598,0.000060,-18.800767,0.000033,0,1,1,1
4,10000,1986-06,13007.0,NaN,OPTIMUM MANUFACTURING -CL A,1.284,0.578,0.395,0.706,-0.484,...,-0.000412,-0.000041,0.000601,0.000060,-18.819512,0.000034,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600003,93436,2019-08,184996.0,1318605.0,TESLA INC,31872.597,5715.393,5082.746,24722.136,-408.334,...,-0.000100,-0.000010,0.006070,0.000611,-13.345812,0.000126,0,0,0,0
2600004,93436,2019-09,184996.0,1318605.0,TESLA INC,31872.597,5715.393,5082.746,24722.136,-408.334,...,-0.000094,-0.000009,0.005661,0.000570,-13.291949,0.000117,0,0,0,0
2600005,93436,2019-10,184996.0,1318605.0,TESLA INC,31872.597,5715.393,5082.746,24722.136,-408.334,...,-0.000072,-0.000007,0.004331,0.000435,-13.042239,0.000090,0,0,0,0
2600006,93436,2019-11,184996.0,1318605.0,TESLA INC,32795.000,6040.000,5571.000,25313.000,143.469,...,0.000024,0.000002,0.004234,0.000425,-13.028246,0.000094,0,0,0,0


### Process Data

In [4]:
# process firm data
desired_columns = ['PERMNO', 'date_month', 'GVKEY', 'cik', 'conm', 'atq', 'ceqq', 'cheq', 'ltq', 'niq', 'dlrsn', 'dldte', 'PRC', 'RET', 'SHROUT', 'CFACPR', 'vwretdSP500', 'totvalSP500', 'SIGMA', 'EXRET', 'bankruptcyIndicator']
df = df[desired_columns]
df['PRC'] = abs(df['PRC'])
df['TA_adj'] = (df['atq'] + 0.1 * ((df['PRC'] * df['SHROUT']) - df['ceqq']))
df['MKTCAP'] = df['PRC'] * df['SHROUT']
df['NITA'] = df['niq'] / df['TA_adj']
df['NIMTA'] = df['niq'] / (df['MKTCAP'] + df['ltq'])
df['TLTA'] = df['ltq'] / df['TA_adj']
df['TLMTA'] = df['ltq'] / (df['MKTCAP'] + df['ltq'])
df['RSIZE'] = np.log10(df['MKTCAP'] / df['totvalSP500'])
df['CASHMTA'] = df['cheq'] / (df['MKTCAP'] + df['ltq'])


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [5]:
# Compute financial distress indicator
df['financialDistress'] = (((df.groupby('PERMNO')['RET'].tail(DEPENDENT_VARIABLE_WINDOW) < FINANCIAL_DISTRESS_THRESHOLD) & (df.groupby('PERMNO')['dlrsn'].tail(DEPENDENT_VARIABLE_WINDOW) > 0)))
df['financialDistress'] = df['financialDistress'].map({np.nan: 0, False: 0, True: 1})
maxFinancialDistress = df.groupby('PERMNO')['financialDistress'].max().reset_index().rename(columns={'financialDistress': 'everFinanciallyDistressed'})
df = pd.merge(df, maxFinancialDistress, left_on=['PERMNO'], right_on=['PERMNO'])
df['financialDistress'] = df['everFinanciallyDistressed']
df['financialDistress'] = np.where(df['PERMNO'] != df['PERMNO'].shift(-1), df['financialDistress'], 0)
df['combined'] = df[['bankruptcyIndicator', 'financialDistress']].max(axis=1)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
# Impute missing, infinity, and negative infinity using forward fill
imputation_columns = ['NITA', 'NIMTA', 'TLTA', 'TLMTA', 'RSIZE', 'CASHMTA', 'EXRET', 'SIGMA']
for col in imputation_columns:
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)
    df[col] = df.groupby('PERMNO')[col].fillna(method='ffill')

In [7]:
df['year'] = pd.to_datetime(df['date_month'].apply(lambda x: str(x)[:4]))

In [8]:
def rev_roll(x):
    return x.iloc[::-1].rolling(DEPENDENT_VARIABLE_WINDOW, min_periods=0).max().iloc[::-1]

df[f'financialDistress_{DEPENDENT_VARIABLE_WINDOW}_periods'] = df.groupby('PERMNO')['combined'].transform(rev_roll)

In [9]:
# clear 2019/2020
df = df[df['year'] < '1/1/2019']

In [10]:
df.to_csv('DJJX-data-2.csv', index=False)

In [ ]:
# shift returns for portfolios
df = pd.read_csv('DJJX-data-2.csv', low_memory=False)
permno = pd.Series(df['PERMNO'].unique()).sort_values().reset_index(drop=True)
d = pd.DataFrame()
for n in permno:
    t = df[df['PERMNO'] == n].reset_index(drop=True)
    t['RET-shifted'] = t['RET'].shift(-1) # shift ret_+1 to ret_0
    d = d.append(t)
    
df = d.reset_index(drop=True)
df.to_csv('DJJX-data-shifted.csv', index=False)

In [3]:
# adjust to decimale scale and get datetime feature
ff3 = pd.read_csv('ff3.csv')
ff3.iloc[:,1:] = ff3.iloc[:,1:].apply(lambda x: x/100) # adjust to decimle scale
ff3['date_month'] = pd.to_datetime(ff3['Date'],format='%Y%m').dt.to_period('M')
ff3['year'] = ff3['date_month'].dt.year # add year factor
ff3.to_csv('ff3-clean.csv', index=False)

In [7]:
data = pd.read_csv('DJJX-data-shifted.csv', low_memory=False)

# isolate target and features only
y_col = 'financialDistress_12_periods'
cols = ['date_month','PERMNO','RET','RET-shifted','MKTCAP','NITA','NIMTA','TLTA','TLMTA',
        'CASHMTA','EXRET','RSIZE','PRC','SIGMA',y_col]
d = data[cols].copy()

# replace lagging nans
mask = (~data['RET-shifted'].isna()) & (data['financialDistress_12_periods'] == 1)
median = data[mask]['RET-shifted'].median() # median < mean 
mask = (data['RET-shifted'].isna()) & (data['financialDistress_12_periods'] == 1)
d['RET-shifted'] = np.where(mask, median, data['RET-shifted'])

# drop nan, reset datetimes, add additional feature, & save
d = d.dropna()
d['year'] = pd.to_datetime(d['date_month']).dt.to_period('Y')
d['date_month'] = pd.to_datetime(d['date_month']).dt.to_period('M')
d['SIGMA_TLTA'] = d['SIGMA']*d['TLTA']
d.to_csv('data-clean.csv', index=False)

In [9]:
# exclude top 1000 firms by market cap
d = pd.read_csv('data-clean.csv', low_memory=False)
months = pd.Series(d['date_month'].unique()).sort_values().reset_index(drop=True)
exclude_df = pd.DataFrame()
for month in months:
    t = d[d['date_month'] == month].reset_index(drop=True)
    t = t.sort_values('MKTCAP')[0:-1000]
    exclude_df = exclude_df.append(t)
    
exclude_df = exclude_df.reset_index(drop=True)
exclude_df.to_csv('data-exclude.csv', index=False)

In [10]:
d

,date_month,PERMNO,RET,RET-shifted,MKTCAP,NITA,NIMTA,TLTA,TLMTA,CASHMTA,EXRET,RSIZE,PRC,SIGMA,financialDistress_12_periods,year,SIGMA_TLTA
0,1972-02,30082,-0.017544,0.107143,1211.000,-0.003030,-0.000309,0.012471,0.001270,0.000100,-0.047473,-8.656905,3.500,0.824265,0.0,1972,0.010279
1,1972-02,37073,0.150000,0.326087,1604.250,-0.002899,-0.000296,0.019808,0.002024,0.000106,0.109989,-8.534777,2.875,0.829627,0.0,1972,0.016434
2,1972-02,37276,-0.161290,0.153846,2031.250,-0.001825,-0.000185,0.007718,0.000782,0.000159,-0.205664,-8.432286,3.250,0.665981,0.0,1972,0.005140
3,1972-02,37815,0.051282,-0.195122,2044.875,-0.000431,-0.000044,0.019893,0.002050,0.000106,0.020237,-8.429382,5.125,0.756233,0.0,1972,0.015044
4,1972-02,29073,-0.083333,-0.030303,2095.500,-0.000038,-0.000004,0.000603,0.000061,0.000866,-0.116784,-8.418761,4.125,0.738219,0.0,1972,0.000445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335502,2018-11,10812,0.065742,-0.117108,1699985.250,0.000112,0.000011,0.001279,0.000128,0.000073,0.044112,-7.150420,20.750,0.244548,0.0,2018,0.000313
1335503,2018-11,14472,-0.051744,-0.086514,1706318.150,-0.000043,-0.000004,0.000795,0.000080,0.000093,-0.072690,-7.148805,57.910,0.515630,0.0,2018,0.000410
1335504,2018-11,16434,-0.059353,-0.077899,1708266.000,0.000041,0.000004,0.004646,0.000469,0.000039,-0.080747,-7.148310,22.850,0.253385,0.0,2018,0.001177
1335505,2018-11,17234,-0.051766,-0.064868,1711999.350,0.001500,0.000156,0.031499,0.003277,0.000408,-0.072713,-7.147362,15.570,0.361124,0.0,2018,0.011375
